# Imports y definiciones

In [4]:
import os
import json
import numpy as np
import pandas as pd
from ast import literal_eval
from collections import defaultdict

# Revisión de componentes de las bebidas energéticas

In [5]:
# Poblando el diccionario de información
energyzer_fold = 'bebidas'
energyzer_files = [f'{energyzer_fold}/{i}' for i in os.listdir(energyzer_fold)]
energyzer_dict = defaultdict(list)

for name in energyzer_files:
    with open(name, 'r', encoding='utf8') as file:
        value_total = 0
        
        # Obtener la dosis
        dosis = file.readline().strip().split(';')
        
        # Saltar la primera linea
        file.readline()
        
        # Agregar los nombres
        energyzer_dict['"Name"'].append(name)
        
        # Modificando la dosis total
        if dosis[1] == 'g':
            dosis_total = float(dosis[0])
        elif dosis[1] == 'mg':
            dosis_total = float(dosis[0]) / 1e3
        elif dosis[1] == 'ug':
            dosis_total = float(dosis[0]) / 1e6
        
        # Agregando la dosis al registro
        energyzer_dict['"Size"'].append(dosis_total)
        
        # Revisando los datos
        for line in file:
            data_line = line.strip().split(';')
            
            # Obtención del gramaje
            gram_unit = data_line[2].lower()
            
            # Modificando el gramaje
            if gram_unit == 'g':
                value_to = float(data_line[1])
            elif gram_unit == 'mg':
                value_to = float(data_line[1]) / 1e3
            elif gram_unit == 'ug':
                value_to = float(data_line[1]) / 1e6
            
            # Normalizando por la dosis total
            value_perc = value_to / dosis_total
            
            # Agregando al diccionario
            energyzer_dict[data_line[0]].append(value_perc)

In [8]:
energyzer_dict

defaultdict(list,
            {'"Name"': ['bebidas/energy_drink.csv',
              'bebidas/energy_drink_full_throttle.csv',
              'bebidas/energy_drink_monster.csv',
              'bebidas/energy_drink_mountain_dew_amp.csv',
              'bebidas/energy_drink_nos.csv',
              'bebidas/energy_drink_no_fear.csv',
              'bebidas/energy_drink_no_fear_motherload.csv',
              'bebidas/energy_drink_ocean_spray_cranenergy_juice_drink.csv',
              'bebidas/energy_drink_red_bull.csv',
              'bebidas/energy_drink_rockstar.csv',
              'bebidas/energy_drink_sobe_energize_energy_juice_drink.csv',
              'bebidas/energy_drink_vault.csv',
              'bebidas/energy_drink_xs.csv',
              'bebidas/energy_drink_xs_gold_plus.csv'],
             '"Size"': [248.0,
              248.0,
              31.0,
              248.0,
              248.0,
              248.0,
              248.0,
              248.0,
              248.0,
       

In [9]:
# Definición de las componentes que aportan
interest_compounds = list()

for key in energyzer_dict.keys():
    if key != '"Name"':
        # Si es que al menos tiene algo que aportar, se agrega
        if sum(energyzer_dict[key]) > 0:
            interest_compounds.append(key)

In [10]:
interest_compounds

['"Size"',
 '"Fat"',
 '"Protein"',
 '"Carbohydrate"',
 '"Water"',
 '"Caffeine"',
 '"Sugars"',
 '"Calcium, Ca"',
 '"Iron, Fe"',
 '"Magnesium, Mg"',
 '"Phosphorus, P"',
 '"Potassium, K"',
 '"Sodium, Na"',
 '"Zinc, Zn"',
 '"Copper, Cu"',
 '"Selenium, Se"',
 '"Vitamin C"',
 '"Thiamin"',
 '"Riboflavin"',
 '"Niacin"',
 '"Vitamin B6"',
 '"Folate"',
 '"Vitamin B12"',
 '"Choline"',
 '"Vitamin K"',
 '"Folic acid"',
 '"Folate, DFE"',
 '"Vitamin B12, added"',
 '"Net carbs"']

# Obtener macro categorías para eliminarlas

In [11]:
macro_categories = set()
with open('bebidas/energy_drink.csv', 'r', encoding='utf8') as file:
    # Saltarse las 2 primeras líneas
    file.readline()
    file.readline()
    
    for line in file:
        info_to_append = line.strip().split(';')[-2:]
        macro_categories.add(info_to_append[0])
        macro_categories.add(info_to_append[1])
        
# Se agregan los net carbs porque son un resumen de los carbohidratos
macro_categories.add('Net carbs')
macro_categories.add('Carbohydrate')

# Se agregan los folatos, porque es lo mismo que acido fólico
macro_categories.add('Folate')
print(macro_categories)

['-', '-']
['-', '-']
['-', '-']
['-', '-']
['-', '-']
['-', '-']
['-', '-']
['-', '-']
['-', '-']
['-', '-']
['-', '-']
['-', '-']
['-', '-']
['-', '-']
['-', '-']
['-', '-']
['-', '-']
['-', '-']
['Vitamin A, RAE', '-']
['-', '-']
['Vitamin A, RAE', '-']
['Vitamin A, RAE', '-']
['-', '-']
['-', '-']
['Vitamin A, RAE', '-']
['Vitamin A, RAE', '-']
['Vitamin A, RAE', '-']
['-', '-']
['-', '-']
['-', '-']
['-', '-']
['-', '-']
['-', '-']
['-', '-']
['-', '-']
['-', '-']
['Folate, DFE', '-']
['Folate, DFE', '-']
['-', '-']
['Vitamin E (alpha-tocopherol)', '-']
['Vitamin B12', '-']
['-', '-']
['Fat', '-']
['Fat', 'Saturated fatty acids']
['Fat', 'Saturated fatty acids']
['Fat', 'Saturated fatty acids']
['Fat', 'Saturated fatty acids']
['Fat', 'Saturated fatty acids']
['Fat', 'Saturated fatty acids']
['Fat', 'Saturated fatty acids']
['Fat', 'Saturated fatty acids']
['-', '-']
['Fat', 'Monounsaturated fatty acids']
['-', '-']
['Fat', 'Monounsaturated fatty acids']
['Fat', 'Monounsaturated f

# Obtener los códigos de los componentes de interés

In [6]:
# Lectura del excel de codificación de las componentes
code_data = pd.read_excel('drink_comps_codes.xlsx')

# Limpiar los datos del archivo antes de partir
with open('interest_drink_comps_codes.csv', 'w', encoding='utf8') as file:
    pass

for i in range(code_data.shape[0]):
    if (f'\"{code_data["Nombre del componente"][i]}\"' in interest_compounds
        and code_data["Nombre del componente"][i] not in macro_categories):
        with open('interest_drink_comps_codes.csv', 'a', encoding='utf8') as file:
            file.write(code_data.loc[[i]].to_csv(sep=';', header=False, index=False).strip())
            file.write('\n')

# Filtro de componentes de interés

In [7]:
# Definición del diccionario de salida
interest_energyzer_dict = dict()

# Definición de los parámetros de interés
interest_keys = list()

with open('interest_drink_comps_codes.csv', 'r', encoding='utf8') as file:
    for line in file:
        interest_keys.append(f'\"{line.strip().split(";")[0]}\"')

# Para cada key, revisar si es que está en la lista y agregarlo al diccionario
# de interés
for key in energyzer_dict.keys():
    if key in ['"Name"', '"Size"'] or key in interest_keys:
        interest_energyzer_dict[key] = energyzer_dict[key]

# Test de sanidad para gramajes por bebida

In [8]:
for num, name in enumerate(interest_energyzer_dict['"Name"']):
    keys_to_rev = list(interest_energyzer_dict.keys())[2:]
#     print(keys_to_rev)
    
    # Cantidad de gramos por componentes
    grams_drink = 0
    
    for key in keys_to_rev:
        grams_drink += interest_energyzer_dict[key][num]
        
    print(name, grams_drink)

bebidas/energy_drink.csv 1.0001171748387099
bebidas/energy_drink_full_throttle.csv 0.9971819883870967
bebidas/energy_drink_monster.csv 0.996079283870968
bebidas/energy_drink_mountain_dew_amp.csv 0.9972590506854838
bebidas/energy_drink_nos.csv 0.9982144846774194
bebidas/energy_drink_no_fear.csv 0.9881215502419353
bebidas/energy_drink_no_fear_motherload.csv 0.9881215502419353
bebidas/energy_drink_ocean_spray_cranenergy_juice_drink.csv 1.0000881504032255
bebidas/energy_drink_red_bull.csv 1.0001171748387099
bebidas/energy_drink_rockstar.csv 0.9974657514112905
bebidas/energy_drink_sobe_energize_energy_juice_drink.csv 0.9893096774193548
bebidas/energy_drink_vault.csv 0.9976112157258066
bebidas/energy_drink_xs.csv 0.9874392880645161
bebidas/energy_drink_xs_gold_plus.csv 0.9874392880645161


## OJO

La suma de la masa de los componentes no dan exactos con la porción total a analizar. 
**Por revisar si es que hay componentes que se forman como suma de otros.**

# Registro en archivo

In [9]:
with open('summary/summary_drinks_dict.txt', 'w', encoding='utf8') as file:
    file.write(str(interest_energyzer_dict))

# Definición de la tabla objetivo

In [10]:
with open('summary/objective_table.csv', 'w', encoding='utf8') as file:
    file.write('Compound;proportion\n')
    
    for key in interest_energyzer_dict.keys():
        if key not in ['"Name"', '"Size"']:
            mean_value = np.mean(interest_energyzer_dict[key])
            file.write(f'{key};{mean_value}\n')